In [1]:
import tensorflow as tf
import os                  # dealing with directories
from tqdm import tqdm
from tensorflow.python import ipu

import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization,ZeroPadding2D
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.python.keras.optimizer_v2.adam import Adam
from tensorflow.python.keras.layers import Concatenate
from pytictoc import TicToc
print(tf.__version__)

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-e_zy3g2a because the default path (/home/ipuuser/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


2.4.4


In [1]:
import gcipuinfo

In [5]:
ipu_info = gcipuinfo.gcipuinfo()
num_devices = len(ipu_info.getDevices())

In [6]:
device_powers = ipu_info.getNamedAttributeForAll(gcipuinfo.TotalBoardPower)

In [8]:
device_powers.

gcipuinfo_py3.StringVector

In [2]:
num_ipus = 1
t = TicToc() # create TicToc instance
t.tic() # Start timer

In [3]:
# Configure the IPU system
ipu_config = ipu.config.IPUConfig()
ipu_config.auto_select_ipus = num_ipus
ipu_config.configure_ipu_system()

In [4]:
path='train'

In [5]:
# Create an execution strategy.
strategy = ipu.ipu_strategy.IPUStrategy()

In [6]:
labels=[]

## Reading the Data

In [7]:
def image_read(TRAIN_DIR):
    training_data = []
    for c in os.listdir(path):
        for img in tqdm(os.listdir(os.path.join(path,c))):
            img_path= os.path.join(path,c,img)
            image=plt.imread(img_path)
            if image is not None:
                training_data.append(np.array(image))
                labels.append(c)
    return training_data, labels

In [8]:
Fruit_data, labels=image_read(path)

100%|██████████| 478/478 [00:00<00:00, 3444.84it/s]


In [9]:
len(Fruit_data),len(labels)

(2908, 2908)

Converting the Fruit_data and labels object type to Numpy array

In [10]:
Fruit_data,labels=np.array(Fruit_data),np.array(labels)

In [11]:
print("Class names from the dataset {}".format(np.unique(labels)))

Class names from the dataset ['Tomato 3' 'Tomato 4' 'Tomato Cherry Red' 'Tomato Maroon' 'Tomato Yellow'
 'Walnut']


# Data Transformation

In [12]:
from sklearn.preprocessing import LabelEncoder

In [13]:
le=LabelEncoder()

In [14]:
labels_encoded=le.fit_transform(labels)

# Normilization of Image Data

In [15]:
Norm_Fruit_data=Fruit_data/255

# Train and Test split of Data

In [16]:
import sklearn.model_selection as ms
x_train,x_test,y_train,y_test=ms.train_test_split(Norm_Fruit_data,labels_encoded,test_size=0.3,random_state=121123451)

In [17]:
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((2035, 100, 100, 3), (873, 100, 100, 3), (2035,), (873,))

In [18]:
batch_size=5

In [19]:
def make_divisible(number, divisor):
    return number - number % divisor

In [20]:
# Adjust dataset lengths to be divisible by the batch size
train_data_len = x_train.shape[0]
train_data_len = make_divisible(train_data_len, batch_size)
x_train, y_train = x_train[:train_data_len], y_train[:train_data_len]

test_data_len = x_test.shape[0]
test_data_len = make_divisible(test_data_len, batch_size)
x_test, y_test = x_test[:test_data_len], y_test[:test_data_len]

In [21]:
print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

(2035, 100, 100, 3) (870, 100, 100, 3) (2035,) (870,)


In [22]:
input_shape = (100, 100, 3)
num_classes=6

# One Hot Encoding

In [23]:
from tensorflow.keras.utils import to_categorical
y_train_onehot= to_categorical(y_train)
y_test_onehot=to_categorical(y_test)
t.toc('The data loading and preprocessing took')

The data loading and preprocessing took 48.540600 seconds.


In [24]:
s = TicToc() # create TicToc instance
s.tic() # Start timer
with strategy.scope():
    model = ResNet50(weights=None,include_top=False, input_shape=(100,100,3))
    inputs=model.input
    x = model.output
    x = Flatten()(x)
    x=Dense(units=100,activation='relu')(x)
    x=Dropout(0.25)(x)
    x=Dense(6, activation='softmax')(x)
    resnet_model = tf.keras.Model(inputs=inputs, outputs=x)
    resnet_model.compile(loss=tf.keras.losses.categorical_crossentropy,
                optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'], steps_per_execution=50)
    resnet_model.fit(x_train,y_train_onehot,batch_size=5,epochs=50, shuffle=True,steps_per_epoch=100)

INFO:tensorflow:The provided set of data has an unknown size. This can result in runtime errors if not enough data is provided during execution.
Epoch 1/50
100/100 [==============================] - 305s 3s/step - loss: 7.0225 - accuracy: 0.2322
Epoch 2/50
100/100 [==============================] - 2s 16ms/step - loss: 1.8724 - accuracy: 0.2750
Epoch 3/50
100/100 [==============================] - 1s 13ms/step - loss: 1.7857 - accuracy: 0.2289
Epoch 4/50
100/100 [==============================] - 1s 13ms/step - loss: 1.7796 - accuracy: 0.2599
Epoch 5/50
100/100 [==============================] - 1s 13ms/step - loss: 1.7849 - accuracy: 0.2140
Epoch 6/50
100/100 [==============================] - 1s 13ms/step - loss: 1.7705 - accuracy: 0.2939
Epoch 7/50
100/100 [==============================] - 1s 14ms/step - loss: 1.7731 - accuracy: 0.2502
Epoch 8/50
100/100 [==============================] - 1s 13ms/step - loss: 1.7671 - accuracy: 0.2590
Epoch 9/50
100/100 [===========================

In [25]:
s.toc('Resnet 50 Model compile & training took')

Resnet 50 Model compile & training took 475.224874 seconds.
